# Parte 1: Comenzando
## Importación de bibliotecas y configuración de entorno: 
Se importan las bibliotecas necesarias, como TensorFlow, sklearn y numpy. Además, se establece la semilla para la reproducibilidad de los resultados y se configura la variable de entorno 'PATH' para el entorno de Vivado.

In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

2024-05-15 02:18:03.135398: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-15 02:18:03.274620: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Obtención y preparación de datos: 
Este código carga imágenes de entrenamiento y prueba desde carpetas específicas, las redimensiona a un tamaño deseado y las almacena junto con sus etiquetas en matrices numpy para su posterior procesamiento. Los datos se dividen en imagenes (X) y etiquetas (y).

In [2]:
# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

# Librerias de ayuda
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.11.1


In [3]:
!pip install opencv-python
import cv2
import os
import numpy as np

# Tamaño deseado para todas las imágenes
nuevo_ancho = 60
nuevo_alto = 60

# Ruta a la carpeta de imágenes
data_dir = "./signals"

# Función para cargar imágenes y etiquetas de una sola carpeta
def load_images_and_labels(directory):
    images = []
    labels = []
    for folder_name in os.listdir(directory):
        folder_path = os.path.join(directory, folder_name)
        if os.path.isdir(folder_path):
            for image_file in os.listdir(folder_path):
                image_path = os.path.join(folder_path, image_file)
                # Leer la imagen en escala de grises
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
                # Redimensionar la imagen
                image = cv2.resize(image, (nuevo_ancho, nuevo_alto))
                # Normalizar la imagen
                image = image / 255.0
                images.append(image)
                # Usar el nombre de la carpeta como etiqueta
                labels.append(folder_name)
    return np.array(images), np.array(labels)

# Cargar imágenes y etiquetas de la carpeta de datos
images, labels = load_images_and_labels(data_dir)

# Crear un diccionario con los datos
data = {'data': images, 'target': labels}

# Organizar los datos según la estructura deseada
X, y = data['data'], data['target']

# Verificar las dimensiones de los datos cargados
print("Dimensiones de las imágenes:", X.shape)
print("Dimensiones de las etiquetas:", y.shape)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 13.0 MB/s eta 0:00:0000:0100:01
Dimensiones de las imágenes: (40, 60, 60)
Dimensiones de las etiquetas: (40,)


### Imprimir información sobre el conjunto de datos: 
Imprime la forma (shape) del conjunto de datos `X` y las etiquetas `y` utilizando `X.shape` y `y.shape`. Además, imprime las primeras cinco filas de características y etiquetas utilizando `X[:5]` y `y[:5]`.

In [13]:
print(X.shape, y.shape)
print(X[:5])
print(y[:40])


(40, 60, 60) (40,)
[[[0.79215686 0.8627451  0.89411765 ... 0.10588235 0.2745098  0.85098039]
  [0.6745098  0.81176471 0.89019608 ... 0.11764706 0.26666667 0.83137255]
  [0.60784314 0.7372549  0.8745098  ... 0.10980392 0.34901961 0.81960784]
  ...
  [0.4        0.40784314 0.38039216 ... 0.54117647 0.66666667 0.67058824]
  [0.52156863 0.36862745 0.37254902 ... 0.53333333 0.68235294 0.67843137]
  [0.54901961 0.65098039 0.55294118 ... 0.52156863 0.68235294 0.69019608]]

 [[0.22745098 0.19607843 0.14901961 ... 0.8627451  0.39215686 0.8627451 ]
  [0.25490196 0.24313725 0.24705882 ... 0.64705882 0.49803922 0.60784314]
  [0.32156863 0.25882353 0.20392157 ... 0.33333333 0.45490196 0.76862745]
  ...
  [0.10196078 0.05098039 0.37647059 ... 0.4745098  0.35686275 0.55686275]
  [0.23921569 0.11372549 0.43529412 ... 0.62352941 0.77647059 0.81568627]
  [0.18823529 0.15686275 0.49411765 ... 0.6745098  0.83921569 0.88235294]]

 [[0.31764706 0.30980392 0.28235294 ... 0.34509804 0.29411765 0.49803922]
  [

En este bloque de código, se realiza la codificación de las etiquetas `y` para el entrenamiento del modelo. Primero, se utiliza `LabelEncoder()` para transformar las etiquetas de clase de cadena a numérico. Luego, se aplica `to_categorical` para convertir estas etiquetas numéricas en un formato binario "One Hot" para la clasificación multiclase. Después, se divide el conjunto de datos en conjuntos de entrenamiento y prueba utilizando `train_test_split`, con un tamaño de prueba del 20% y una semilla aleatoria de 42 para la reproducibilidad de los resultados.

Además, se realiza el escalado de las características del conjunto de datos utilizando `StandardScaler()` para asegurar que todas las características tengan una media de cero y una desviación estándar de uno, lo que puede mejorar el rendimiento del modelo. 

### Archivos creados:
Se guardan los conjuntos de datos de entrenamiento y prueba, así como las etiquetas, en archivos `.npy`, junto con la clase original de las etiquetas.
* x_train_val.npy
* x_test.npy
* y_train_val.npy
* y_test.npy
* classes.npy

In [14]:
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 5)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(y[:5])

[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]]


In [16]:
from sklearn.preprocessing import StandardScaler

# Aplanar cada imagen
X_train_val_flat = X_train_val.reshape(X_train_val.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Escalar cada dimensión por separado
scaler = StandardScaler()
X_train_val_scaled = scaler.fit_transform(X_train_val_flat)
X_test_scaled = scaler.transform(X_test_flat)

# Remodelar los datos escalados a su forma original
X_train_val_scaled = X_train_val_scaled.reshape(X_train_val.shape)
X_test_scaled = X_test_scaled.reshape(X_test.shape)

In [17]:
np.save('X_train_val.npy', X_train_val)
np.save('X_test.npy', X_test)
np.save('y_train_val.npy', y_train_val)
np.save('y_test.npy', y_test)
np.save('classes.npy', le.classes_)

## Construcción de modelo
Se construye un modelo de red neuronal con tres capas ocultas, cada una con activación ReLU, seguidas de una capa de salida con activación softmax. Las capas ocultas tienen 64, 32 y 32 neuronas respectivamente, mientras que la capa de salida tiene 5 neuronas para clasificar en cinco clases diferentes.

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks

## Entrenando el modelo
Utilizaremos el optimizador Adam con una pérdida de entropía cruzada categórica. Los callbacks disminuirán la tasa de aprendizaje y guardarán el modelo en un directorio llamado 'model_1'. El modelo no es muy complejo, así que esto debería tomar solo unos minutos incluso en la CPU. Si has reiniciado el kernel del notebook después de entrenar una vez, establece `train = False` para cargar el modelo entrenado.

In [30]:
train = True
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Input

# Define the model
model = Sequential([
    Input(shape=(60, 60)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(5, activation='softmax')
])

if train:
    adam = Adam(lr=0.0001)
    model.compile(optimizer=adam, loss=['categorical_crossentropy'], metrics=['accuracy'])
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir='model_1',
    )
    model.fit(
        X_train_val,
        y_train_val,
        batch_size=1024,
        epochs=30,
        validation_split=0.25,
        shuffle=True,
        callbacks=callbacks.callbacks,
    )
else:
    from tensorflow.keras.models import load_model

    model = load_model('model_1/KERAS_check_best_model.h5')

Epoch 1/30
1/1 [==============================] - ETA: 0s - loss: 1.7257 - accuracy: 0.1667
***callbacks***
saving losses to model_1/losses.log

Epoch 1: val_loss improved from inf to 0.83685, saving model to model_1/KERAS_check_best_model.h5

Epoch 1: val_loss improved from inf to 0.83685, saving model to model_1/KERAS_check_best_model_weights.h5

Epoch 1: saving model to model_1/KERAS_check_model_last.h5

Epoch 1: saving model to model_1/KERAS_check_model_last_weights.h5

***callbacks end***

1/1 [==============================] - 0s 368ms/step - loss: 1.7257 - accuracy: 0.1667 - val_loss: 0.8369 - val_accuracy: 0.3750 - lr: 0.0010
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.9297 - accuracy: 0.5000
***callbacks***
saving losses to model_1/losses.log

Epoch 2: val_loss did not improve from 0.83685

Epoch 2: val_loss did not improve from 0.83685

Epoch 2: saving model to model_1/KERAS_check_model_last.h5

Epoch 2: saving model to model_1/KERAS_check_model_last_w

## Evaluacion del rendimiento
Esta sección evalúa el rendimiento del modelo mediante la verificación de la precisión y la creación de una curva ROC.
Se utiliza la función accuracy_score de scikit-learn para calcular la precisión del modelo. Luego, se genera una curva ROC utilizando la función makeRoc del módulo de visualización, lo que proporciona una representación gráfica del rendimiento del modelo en diferentes umbrales de clasificación.

In [31]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

y_keras = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
plt.figure(figsize=(9, 9))
_ = plotting.makeRoc(y_test, y_keras, le.classes_)

1/1 [==============================] - 0s 28ms/step
Accuracy: 1.0


/home/jovyan/HLS4ML/plotting.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'j_index' in labels:


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U5')) -> None

<Figure size 900x900 with 0 Axes>

## Convertir el modelo en firmware FPGA con hls4ml
Ahora pasaremos por los pasos para convertir el modelo que entrenamos en un firmware FPGA optimizado de baja latencia con hls4ml. Primero, evaluaremos su rendimiento de clasificación para asegurarnos de que no hayamos perdido precisión utilizando tipos de datos de punto fijo. Luego, sintetizaremos el modelo con Vivado HLS y verificaremos las métricas de latencia y uso de recursos de la FPGA.

### Crear una configuración y un modelo de hls4ml
La biblioteca de inferencia de redes neuronales hls4ml se controla a través de un diccionario de configuración. En este ejemplo, utilizaremos la variación más simple; en ejercicios posteriores se verán configuraciones más avanzadas.

In [ ]:
import hls4ml

config = hls4ml.utils.config_from_keras_model(model, granularity='model')
print("-----------------------------------")
print("Configuration")
plotting.print_dict(config)
print("-----------------------------------")
hls_model = hls4ml.converters.convert_from_keras_model(
    model, hls_config=config, output_dir='model_1/hls4ml_prj', part='xcu250-figd2104-2L-e'
)

Visualiza la arquitectura del modelo generado por hls4ml, muestra las formas de las capas y los tipos de datos.La visualización se genera utilizando la función plot_model de hls4ml.utils.

In [ ]:
hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

## Compilar, predecir
Ahora necesitamos asegurarnos de que el rendimiento de este modelo siga siendo bueno. Compilamos el hls_model, y luego usamos hls_model.predict para ejecutar el firmware de la FPGA con emulación bit-precisa en la CPU.

In [ ]:
hls_model.compile()
X_test = np.ascontiguousarray(X_test)
y_hls = hls_model.predict(X_test)

## Comparación:
Después de comparar, calculamos la precisión de ambos modelos utilizando la función `accuracy_score` de scikit-learn. Luego, creamos un nuevo gráfico para comparar las curvas ROC de ambos modelos utilizando la función `makeRoc` del módulo `plotting`.

In [ ]:
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))))
print("hls4ml Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_hls, axis=1))))

fig, ax = plt.subplots(figsize=(9, 9))
_ = plotting.makeRoc(y_test, y_keras, le.classes_)
plt.gca().set_prop_cycle(None)  # reset the colors
_ = plotting.makeRoc(y_test, y_hls, le.classes_, linestyle='--')

from matplotlib.lines import Line2D

lines = [Line2D([0], [0], ls='-'), Line2D([0], [0], ls='--')]
from matplotlib.legend import Legend

leg = Legend(ax, lines, labels=['keras', 'hls4ml'], loc='lower right', frameon=False)
ax.add_artist(leg)

## Sintesis
Ahora vamos a utilizar Vivado HLS para sintetizar el modelo. Podemos ejecutar la construcción utilizando un método de nuestro objeto hls_model. Después de ejecutar este paso, podemos integrar el IP generado en un flujo de trabajo para compilarlo para una placa FPGA específica. En este caso, simplemente revisaremos los informes que genera Vivado HLS, verificando la latencia y el uso de recursos.

**Esto puede llevar varios minutos.**

Mientras la Síntesis en C se está ejecutando, podemos monitorear el progreso observando el archivo de registro, abriendo una terminal desde el directorio de inicio del cuaderno y ejecutando:

`tail -f model_1/hls4ml_prj/vivado_hls.log`

In [ ]:
hls_model.build(csim=False)

## Verificar los informes
Imprime los informes generados por Vivado HLS. Presta atención a las secciones de Latencia y 'Estimaciones de utilización'.

In [ ]:
hls4ml.report.read_vivado_report('model_1/hls4ml_prj/')

## Ejercicio
Dado que `ReuseFactor = 1`, esperamos que cada multiplicación utilizada en la inferencia de nuestra red neuronal use 1 DSP. ¿Es esto lo que observamos? (Ten en cuenta que la capa Softmax debería usar 5 DSP, o 1 por clase).
Calcula cuántas multiplicaciones se realizan para la inferencia de esta red...
(Discutiremos el resultado)